In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/salma souissi/Downloads/data_all_cities_hourly.csv")

In [3]:
print(df.shape)

(1042268, 14)


In [4]:
print(df.head())

   temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt    pres  tsun  coco  \
0  16.3   2.3  39.0   NaN   NaN  260.0   7.6   NaN  1023.8   NaN   2.0   
1  15.0   1.1  39.0   NaN   NaN  270.0  13.0   NaN  1023.0   NaN   2.0   
2  15.0  -1.2  33.0   NaN   NaN  260.0  11.2   NaN  1023.0   NaN   2.0   
3  14.1  -1.1  35.0   NaN   NaN  150.0   1.8   NaN  1023.3   NaN   2.0   
4  13.0   1.0  44.0   NaN   NaN  180.0   9.4   NaN  1022.0   NaN   2.0   

         date  hour   city  
0  2020-02-24    18  Tunis  
1  2020-02-24    19  Tunis  
2  2020-02-24    20  Tunis  
3  2020-02-24    21  Tunis  
4  2020-02-24    22  Tunis  


In [5]:
print(df.columns)

Index(['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'tsun', 'coco', 'date', 'hour', 'city'],
      dtype='object')


In [6]:
print(df['city'].unique())

['Tunis' 'Ariana' 'Ben Arous' 'Manouba' 'Nabeul' 'Zaghouan' 'Bizerte'
 'Beja' 'Jendouba' 'Kef' 'Siliana' 'Kasserine' 'Sidi Bouzid' 'Kairouan'
 'Kébili' 'Tozeur' 'Gabès' 'Medenine' 'Tataouine' 'Mahdia' 'Monastir'
 'Sousse' 'Sfax' 'Gafsa']


In [7]:
# Renommer les colonnes
df.rename(columns={  
    'temp': 'temperature',  
    'prcp': 'precipitation',  
    'wspd': 'wind_speed', 
    'rhum':'humidity'
}, inplace=True)
#supprimer les colonnes non necessaires 
df=df.drop(columns=['dwpt','snow','wdir','wpgt','pres','tsun','coco'])
# Nouveau ordre des colonnes
new_order = ['city', 'date', 'hour', 'temperature', 'precipitation', 'wind_speed','humidity']

# Réorganiser les colonnes
df = df[new_order]

# Vérifier le nouveau DataFrame
print(df.head())

    city        date  hour  temperature  precipitation  wind_speed  humidity
0  Tunis  2020-02-24    18         16.3            NaN         7.6      39.0
1  Tunis  2020-02-24    19         15.0            NaN        13.0      39.0
2  Tunis  2020-02-24    20         15.0            NaN        11.2      33.0
3  Tunis  2020-02-24    21         14.1            NaN         1.8      35.0
4  Tunis  2020-02-24    22         13.0            NaN         9.4      44.0


In [8]:
# Remplacer les valeurs NaN par 0
df.fillna(0, inplace=True)

In [12]:
pip install holidays 

  Obtaining dependency information for holidays from https://files.pythonhosted.org/packages/f8/8f/9cff125e50b56e29e7e05776dc74e56fc70b79830f0b85e947e5be831e96/holidays-0.67-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/820.7 kB ? eta -:--:--
   - -------------------------------------- 30.7/820.7 kB 1.3 MB/s eta 0:00:01
   -- ------------------------------------ 61.4/820.7 kB 656.4 kB/s eta 0:00:02
   ---- --------------------------------- 102.4/820.7 kB 653.6 kB/s eta 0:00:02
   ----- -------------------------------- 122.9/820.7 kB 722.1 kB/s eta 0:00:01
   -------- ----------------------------- 184.3/820.7 kB 794.9 kB/s eta 0:00:01
   --------- ---------------------------- 204.8/820.7 kB 731.4 kB/s eta 0:00:01
   ---------- --------------------------- 235.5/820.7 kB 758.5 kB/s eta 0:00:01
   ------------ ------------------------- 266.2/820.7 kB 780.5 kB/s eta 0:00:01
   -------------- ----------------------- 307.2/820.7 kB 759.5 kB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\salma souissi\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [9]:
import holidays
# Définir le pays (ex: 'TN' pour la Tunisie)
jours_feries = holidays.country_holidays('TN')


In [10]:
# Convertir la colonne Date en datetime
df["date"] = pd.to_datetime(df["date"])


In [11]:
# Ajout de la colonne température moyenne sur 7 jours Pour capturer l’inertie des comportements.
df['température_moyenne_7j'] = df['temperature'].rolling(window=7, min_periods=1).mean()

#ajouter une colonne indiquant le mois :
df["mois"] = df["date"].dt.month
#____________________
jours_semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi","Samedi", "Dimanche"]
#Générer les jours en respectant l'ordre
df["Jour"] = [jours_semaine[i % 7] for i in range(len(df))]
#Ajouter la colonne Weekend (1 si Samedi ou Dimanche, sinon 0)
df["Weekend"] = df["Jour"].isin(["Samedi", "Dimanche"]).astype(int)
# Ajouter une colonne indiquant si la date est un jour férié
df['jour_ferie'] = df['date'].apply(lambda x: x in jours_feries).astype(int)
# Ajouter une colonne "saison"
df["saison"] = df["date"].apply(lambda x: "été" if x.month in [6, 7, 8] else "hiver" if x.month in [12, 1, 2] else "printemps/automne")
#vacances scolaires :
vacances_scolaires = [('2020-12-15', '2021-01-01'), ('2021-03-14', '2021-03-27'),('2021-06-30','2021-09-14'),('2021-12-15', '2022-01-01'), ('2022-03-14', '2022-03-27'),('2022-06-30','2022-09-14'),('2022-11-15', '2023-01-1'), ('2023-03-14', '2023-03-27'),('2023-06-30','2023-09-14'),('2023-11-15', '2024-01-1'), ('2024-03-14', '2024-03-27'),('2024-06-30','2024-09-14')]
df['vacances_scolaires'] = df['date'].apply(
    lambda x: 1 if any(pd.to_datetime(start) <= x <= pd.to_datetime(end) for start, end in vacances_scolaires) else 0)
#ramadan :
ramadan_dates = {
    '2020': ('2020-04-23', '2020-05-23'),
    '2021': ('2021-04-13', '2021-05-12'),
    '2022': ('2022-04-02', '2022-05-01'),
    '2023': ('2023-03-23', '2023-04-22'),
    '2024': ('2024-03-11', '2024-04-09')
}
df['ramadan'] = df['date'].apply(
    lambda x: 1 if any(pd.to_datetime(start) <= x <= pd.to_datetime(end) for start, end in ramadan_dates.values()) else 0)
#new year:
new_year_dates = [('2020-12-31', '2020-12-31'), ('2021-12-31', '2021-12-31'), ('2022-12-31', '2022-12-31'),('2023-12-31', '2023-12-31'),('2024-12-31','2024-12-31'), ('2025-12-31','2025-12-31') ]
df['new_year'] = df['date'].apply(
    lambda x: 1 if any(pd.to_datetime(start) == x for start, end in new_year_dates) else 0)
#type region :
regions = {
    'Tunis': 'Hors côtier',
    'Sousse': 'Côtière',
    'Monastir': 'Côtière',
    'Sfax': 'Hors côtier',
    'Kairouan': 'Intérieure',
    'Gafsa': 'Intérieure',
    'Mahdia': 'Côtière',
    'Nabeul': 'Côtière',
    'Tozeur': 'Intérieure',
    'Jendouba': 'Intérieure',
    'Bizerte': 'Côtière',
    'Kasserine': 'Intérieure',
    'Tataouine': 'Intérieure',
    'Ariana':'Hors côtier',
    'Ben Arous':'Hors côtier',
    'Manouba':'Hors côtier',
    'Zaghouan':'Hors côtier',
    'Kébili':'Intérieure',
    'Gabès':'Intérieure',
    'Medenine':'Intérieure',
    'Sidi Bouzid':'Intérieure',
    'Beja':'Intérieure',
    'Kef':'Intérieure',
    'Siliana':'Intérieure'

}

df['Region'] = df['city'].apply(lambda x: regions.get(x, 'Unknown'))
# Définition des mois qui représentent la haute et basse saison touristique
high_season_months = [5,6,7,8]
medium_season_months = [4,9,10,11]
low_season_months = [1,2,3,12]

# Ajout d'une nouvelle colonne qui définit la saison touristique en fonction du mois
def get_season(month):
    if month in high_season_months:
        return 'Haute '
    elif month in medium_season_months:
        return 'Moyenne'
    else:
        return 'Basse '
df['saison_toristique'] = df['mois'].apply(lambda x: get_season(x))




In [12]:
# Sauvegarder le DataFrame en fichier CSV
df.to_csv('features.csv', index=False)

In [13]:
# Création de la colonne 'periode_journee'
def categoriser_periode(hour):
    if 5 <= hour < 8:
        return 'Petit matin (5h-8h)'
    elif 8 <= hour < 12:
        return 'Matin (8h-12h)'
    elif 12 <= hour < 17:
        return 'Midi-Après-midi (12h-17h)'
    elif 17 <= hour < 21:
        return 'Soirée (17h-21h)'
    else:
        return 'Nuit (21h-5h)'

# Application
df['periode_journee'] = df['hour'].apply(categoriser_periode)

In [14]:
print(df.head(10))

    city       date  hour  temperature  precipitation  wind_speed  humidity  \
0  Tunis 2020-02-24    18         16.3            0.0         7.6      39.0   
1  Tunis 2020-02-24    19         15.0            0.0        13.0      39.0   
2  Tunis 2020-02-24    20         15.0            0.0        11.2      33.0   
3  Tunis 2020-02-24    21         14.1            0.0         1.8      35.0   
4  Tunis 2020-02-24    22         13.0            0.0         9.4      44.0   
5  Tunis 2020-02-24    23         12.0            0.0         5.4      54.0   
6  Tunis 2020-02-25     0          9.7            0.0         3.6      59.0   
7  Tunis 2020-02-25     1          7.0            0.0         5.4      71.0   
8  Tunis 2020-02-25     2          9.0            0.0         5.4      66.0   
9  Tunis 2020-02-25     3          9.4            0.0         9.4      62.0   

   température_moyenne_7j  mois      Jour  Weekend  jour_ferie saison  \
0               16.300000     2     Lundi        0       

In [28]:
# Coefficients mensuels pour les ventes de glaces en Tunisie
monthly_glace_coefficients = {
    1: 0.0,  # Janvier - Pas de consommation
    2: 0.0,  # Février - Pas de consommation
    3: 0.0,  # Mars - Pas de consommation
    4: 0.0,  # Avril - Pas de consommation
    5: 0.3,  # Mai - Début de la consommation (30%)
    6: 1.0,  # Juin - Consommation normale (100%)
    7: 1.0,  # Juillet - Consommation normale (100%)
    8: 1.0,  # Août - Consommation normale (100%)
    9: 1.0,  # Septembre - Consommation normale (100%)
    10: 0.3, # Octobre - Fin de saison (30%)
    11: 0.0, # Novembre - Plus de consommation
    12: 0.0  # Décembre - Plus de consommation
}
# Coefficients régionaux des ventes de glaces (uniquement en été)
region_glace_coefficients = {
    'Côtière': 1.5,     # +50% dans les zones côtières
    'Hors côtier': 1.2, # +20% dans les zones hors côtières
    'Intérieure': 1.0   # Pas d'effet
}
# Coefficients des jours spéciaux
special_day_coefficients = {
    'Weekend': 1.4,   # +40% le samedi/dimanche
    'jour_ferie': 1.6, # +60% les jours fériés
    'vacances_scolaires': 1.5   # +50% pendant les vacances scolaires
}
#l'impact de la température sur les ventes de glaces :
def temperature_glace_impact(temp):
    if temp < 20:
        return 0  # Aucun impact en dessous de 20°C
    elif 20 <= temp < 30:
        return 0.02 * (temp - 20)  # +2% par degré entre 20°C et 30°C
    elif 30 <= temp <35 :
        return 0.05 * (temp - 30) #+5% par degré entre 30°C et 35°C
    elif 35 <= temp < 40:
        return 0.5
    else:
        return -0.3  # Canicule : -30% ,une chaleur extrême (>40°C) peut réduire les sorties.
def heure_glace_impact(hour):
    if 5 <= hour < 8:
        return 0.2  # Faible consommation tôt le matin
    elif 8 <= hour < 12:
        return 0.8  # Consommation modérée
    elif 12 <= hour < 17:
        return 1.0  # Consommation normale
    elif 17 <= hour < 21:
        return 1.5  # Pic de consommation après 17h
    else:
        return 0.6  # Retour à une consommation réduite la nuit



def generer_ventes_glaces(df, base_ventes,bruit=True, bruit_intensite=0.1):
    # Calcul de l'impact des jours spéciaux
    def jour_impact(row):
        coeff = 1.0
        if row['jour_ferie']:
            coeff *= special_day_coefficients['jour_ferie']
        if row['Weekend']:
            coeff *= special_day_coefficients['Weekend']
        if row['vacances_scolaires']:
            coeff *= special_day_coefficients['vacances_scolaires']
        coeff = min(2.0, coeff)  # Plafond à +100%
        return coeff 

    # Calcul des ventes
    def calcul_ventes(row):
        saison = monthly_glace_coefficients[row['mois']]
        region = region_glace_coefficients[row['Region']]
        jour = jour_impact(row)
        temp = 1 + temperature_glace_impact(row['temperature'])
        heure = heure_glace_impact(row['hour'])  
        ventes = base_ventes * saison * region * jour * temp * heure
        return int(np.round(ventes))
        # Ajout de bruit réaliste
        if bruit:
            bruit_gaussien = np.random.normal(0, bruit_intensite * ventes)  # Bruit normal
            bruit_poisson = np.random.poisson(ventes * 0.05)  # Bruit Poissonien (lambda = 5% des ventes)
            ventes = max(0, int(np.round(ventes + bruit_gaussien + bruit_poisson))) #Pas de ventes négatives !
        return ventes

    # Application sur le DataFrame
    df['ventes_glaces'] = df.apply(calcul_ventes, axis=1)

    return df

# Appel de la fonction
df= generer_ventes_glaces(df, base_ventes=100)







In [30]:
print(df['ventes_glaces'].unique())

[  0  35  43  14  12  48  49  51  65  81  66  63  95 114 110  88  22  30
   7  44  33  32  42  56  58  54  10  31  41  40  59  79  45  39  38  52
  77  11  29  50  62   8  36  85  80  55  53  37  76  57  83  34  60  23
  24  25  78  15  72 100  93 112  86  64  61  90 108  46 101 143 148 106
 134 132 181 262 184 180 102 104 188 185 130 191 186 252 151 137 277 274
 187  75 105 103  73  26 159  96 115 144 193 176 126 205 202 109  74  98
 194 268 142 113 139 198 189 206 201 272 267  27 155 141 192 282 136 135
 257 128 258 168 199 107  99 161 190 209 283 140 120 111 150 153 195 287
 124 138 266 211 273 154 197 196 200 292 203 121 295 129 178 212  97 145
 157 204 123  28 378  84 118 117  82 214 298 156 297 208 127 286 173 160
 122 285 170 158 133 125 210 131 207 290 116 177 228 230 264 302 337 323
 221 183 288 360 346 317 432 396 300 334 328 164 425 418 174 182 149 260
 167 163 166  94 216 301 119 291 236 331 310 340 261 284 280 279 169 278
 179 276 175 270  47  92  87  67 269 172 408 223 17

In [31]:
# Sauvegarder le DataFrame en fichier CSV
df.to_csv('features.csv', index=False)

In [43]:
import pandas as pd
import numpy as np

# Coefficients mensuels pour les boissons fraîches
monthly_boisson_coefficients = {
    1: 0.3,  # Janvier - Consommation basse
    2: 0.4,  # Février
    3: 0.5,  # Mars
    4: 0.6,  # Avril
    5: 1.2,  # Mai - Début de saison
    6: 1.5,  # Juin
    7: 2.0,  # Juillet - Pic estival
    8: 2.3,  # Août
    9: 1.5,  # Septembre
    10: 1.0, # Octobre
    11: 0.6, # Novembre
    12: 0.3  # Décembre
}

# Coefficients régionaux
region_boisson_coefficients = {
    'Côtière': 1.7,     # +70% pour les zonnes toristiques 
    'hors côtièr': 1.5,  # +50% plus de population
    'Urbaine': 1.0,     # pas d'impact
}

# Impact des jours spéciaux
special_day_coefficients = {
    'Weekend': 1.3,    # +30%
    'jour_ferie': 1.3, # +40%
    'vacances_scolaires': 1.0,  #neutre
    'ramadan': 2.5     # pic
}

def temperature_boisson_impact(temp, humidity):
    """Impact combiné température/humidité (indice de chaleur)"""
    if temp < 10:
        return -0.5  # -50% en dessous de 10°C
    elif 10 <= temp < 20:
        return 0.01 * (temp - 10)  # +1% par °C
    elif 20 <= temp < 30:
        return 0.03 * (temp - 20) + 0.1  # +3% par °C + base 10%
    else:
        # Formule simplifiée d'indice de chaleur
        hi_effect = 0.03 * temp + 0.001 * humidity * (temp - 14.5)
        return min(hi_effect, 1.0)  # Plafonné à +100%

def heure_boisson_impact(hour, saison):
    """Variation horaire saisonnière"""
    if saison == 'été' :
        if 12 <= hour < 15:
            return 2.0  # Déjeuner
        elif 15 <= hour < 19:
            return 0.8  # Après-midi/soirée
        elif 20<= hour<22 :
            return 1.5
        elif 0 < hour <12 :
            return 0.1
        else:
            return 0.5
    else:
        if 12 <= hour < 14:
            return 1.5  # Heure du déjeuner
        else:
            return 1.0

def generer_ventes_boissons(df, base_ventes=100):
    def calcul_ventes(row):
        
            # Coefficients de base
            mois_coeff = monthly_boisson_coefficients.get(row['mois'], 1.0)
            
            # Impact régional UNIQUEMENT si saison touristique
            region_coeff = 1.0
            if row['saison_toristique'] == 'Oui':
                region_coeff = region_touristique_coefficients.get(row['Region'], 1.0)
            
            # Impact météo
            temp_coeff = 1 + temperature_boisson_impact(row['temperature'], row['humidity'])
            
            # Impact temporel
            heure_coeff = heure_boisson_impact(row['hour'], row['saison'])
            
            # Jours spéciaux
            special_coeff = 1.0
            if row['jour_ferie']:
                special_coeff *= special_day_coefficients['jour_ferie']
            if row['Weekend']:
                special_coeff *= special_day_coefficients['Weekend']
            if row['vacances_scolaires']:
                special_coeff *= special_day_coefficients['vacances_scolaires']
            if row['ramadan'] and (18 <= row['hour'] <= 23):
                special_coeff *= special_day_coefficients['ramadan']
            
            # Calcul final:
            ventes = base_ventes * mois_coeff * region_coeff * temp_coeff * heure_coeff * special_coeff
            return int(np.round(ventes))
            # Ajout de bruit réaliste
            if bruit:
                bruit_gaussien = np.random.normal(0, bruit_intensite * ventes)  # Bruit normal
                bruit_poisson = np.random.poisson(ventes * 0.05)  # Bruit Poissonien (lambda = 5% des ventes)
                ventes = max(0, int(np.round(ventes + bruit_gaussien + bruit_poisson))) #Pas de ventes négatives !
            return ventes
    df['ventes_boissons'] = df.apply(calcul_ventes, axis=1)
    return df
df = generer_ventes_boissons(df, base_ventes=150)   


In [44]:
print(df.columns)

Index(['city', 'date', 'hour', 'temperature', 'precipitation', 'wind_speed',
       'humidity', 'température_moyenne_7j', 'mois', 'Jour', 'Weekend',
       'jour_ferie', 'saison', 'vacances_scolaires', 'ramadan', 'new_year',
       'Region', 'saison_toristique', 'periode_journee', 'ventes_glaces',
       'ventes_boissons'],
      dtype='object')


In [45]:
print(df[['temperature', 'hour', 'Region', 'ventes_boissons']])

         temperature  hour       Region  ventes_boissons
0               16.3    18  Hors côtier               64
1               15.0    19  Hors côtier               63
2               15.0    20  Hors côtier               63
3               14.1    21  Hors côtier               62
4               13.0    22  Hors côtier               62
...              ...   ...          ...              ...
1042263         17.0    13   Intérieure              125
1042264         17.2    14   Intérieure               84
1042265         16.9    15   Intérieure               64
1042266         16.7    16   Intérieure               64
1042267         15.7    17   Intérieure               63

[1042268 rows x 4 columns]


In [46]:
print(df['ventes_boissons'].unique())

[  64   63   62 ... 1167 1190  980]


In [47]:
# Sauvegarder le DataFrame en fichier CSV
df.to_csv('features.csv', index=False)

In [56]:
def impact_vent(wind_speed):
    """Réduction des ventes quand le vent est fort (>30 km/h) , moins de sortir"""
    if wind_speed <= 20:
        return 1.0  # Pas d'impact
    elif 20 < wind_speed <= 40:
        return 1 - 0.01 * (wind_speed - 20)  # -1% par km/h au-dessus de 20
    else:
        return 0.8  # Plafonné à -20%
def impact_precipitation(precip):
    if precip <= 20:
        return max(0.3, 1 - 0.02 * precip)  # Réduction linéaire jusqu'à 20 mm
    else:
        return 0.6  # Plafond à 60% de réduction pour les fortes pluies
def ajuster_ventes(df,colonne_ventes="ventes"):
    def calcul_ventes(row):
        # Nouveaux coefficients météo
        vent_coeff = impact_vent(row['wind_speed'])
        pluie_coeff = impact_precipitation(row['precipitation'])
        
        # Combinaison avec effets multiplicatifs
        ventes =  row[colonne_ventes]* vent_coeff * pluie_coeff
        
        # Bruit aléatoire
        ventes *= np.random.uniform(0.95, 1.05)
        
        return int(np.round(ventes))
    
    df[colonne_ventes] = df.apply(calcul_ventes, axis=1)
    return df


In [57]:
df = ajuster_ventes(df,colonne_ventes="ventes_glaces") 

In [58]:
df = ajuster_ventes(df,colonne_ventes="ventes_boissons") 

In [59]:
# Sauvegarder le DataFrame en fichier CSV
df.to_csv('features.csv', index=False)